In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
store = pd.read_csv('stores.csv')

In [4]:
store.head()

,Store,Date,IsHoliday,Weekly_Sales,Size,Temperature,Fuel_Price,Unemployment,CPI,Average_earning,Population,Population_over_65,Population_under_18
0,1,12/02/2010,True,1641957.44,151315,3.616667,2.548,8.106,211.242170,63840,18191,4779,2170
1,1,19/02/2010,False,1611968.17,151315,4.405556,2.514,8.106,211.289143,63909,18086,4719,2190
2,1,26/02/2010,False,1409727.59,151315,8.127778,2.561,8.106,211.319643,63897,18153,4762,2169
3,1,12/03/2010,False,1439541.59,151315,14.327778,2.667,8.106,211.380643,63465,18308,4695,2181
4,1,19/03/2010,False,1472515.79,151315,12.544444,2.720,8.106,211.215635,63808,18299,4707,2182


In [6]:
store.isnull().any()

Store                  False
Date                   False
IsHoliday              False
Weekly_Sales           False
Size                   False
Temperature            False
Fuel_Price             False
Unemployment           False
CPI                    False
Average_earning        False
Population             False
Population_over_65     False
Population_under_18    False
dtype: bool

In [8]:
store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4545 entries, 0 to 4544
Data columns (total 13 columns):
Store                  4545 non-null int64
Date                   4545 non-null object
IsHoliday              4545 non-null bool
Weekly_Sales           4545 non-null float64
Size                   4545 non-null int64
Temperature            4545 non-null float64
Fuel_Price             4545 non-null float64
Unemployment           4545 non-null float64
CPI                    4545 non-null float64
Average_earning        4545 non-null int64
Population             4545 non-null int64
Population_over_65     4545 non-null int64
Population_under_18    4545 non-null int64
dtypes: bool(1), float64(5), int64(6), object(1)
memory usage: 430.6+ KB


In [35]:
y = store['Weekly_Sales']
X = store.copy()
del X['Weekly_Sales']
del X['Date']
del X['Store']

In [42]:
X.head()

,IsHoliday,Size,Temperature,Fuel_Price,Unemployment,CPI,Average_earning,Population,Population_over_65,Population_under_18
0,True,151315,3.616667,2.548,8.106,211.242170,63840,18191,4779,2170
1,False,151315,4.405556,2.514,8.106,211.289143,63909,18086,4719,2190
2,False,151315,8.127778,2.561,8.106,211.319643,63897,18153,4762,2169
3,False,151315,14.327778,2.667,8.106,211.380643,63465,18308,4695,2181
4,False,151315,12.544444,2.720,8.106,211.215635,63808,18299,4707,2182


In [63]:
std_scaler = sklearn.preprocessing.StandardScaler()
X_std = std_scaler.fit_transform(X)

In [64]:
import sklearn.model_selection
(X_train, X_test, y_train, y_test) = sklearn.model_selection.train_test_split (X_std, y, 
                                                                               train_size=0.7, random_state=8)

## Try Huber 

In [65]:
import sklearn.linear_model
huber = sklearn.linear_model.HuberRegressor()
huber.fit(X_train, y_train)
prediction = huber.predict(X_test)
huber.score(X_test,y_test)

0.86710326146984651

In [66]:
sklearn.model_selection.cross_val_score(huber, X_std, y, cv = 10)

array([ 0.83498132,  0.8793269 ,  0.5454591 ,  0.78608263,  0.79139617,
        0.78571181,  0.89797302,  0.82638635,  0.86491056,  0.8304601 ])

In [69]:
np.mean(sklearn.model_selection.cross_val_score(huber, X_std, y, cv = 10))

0.80426879680475649

In [106]:
huber_coefs = pd.DataFrame({'variable':X.columns, 'coef':huber.coef_, 'abs_coef':np.abs(huber.coef_)})
huber_coefs.sort_values('abs_coef', inplace=True, ascending=False)
huber_coefs

,abs_coef,coef,variable
7,359671.303892,359671.303892,Population
1,97469.448391,97469.448391,Size
9,64892.106555,64892.106555,Population_under_18
5,26146.590052,-26146.590052,CPI
4,16583.326830,-16583.326830,Unemployment
0,13311.684233,13311.684233,IsHoliday
6,11051.700871,11051.700871,Average_earning
8,7941.643378,7941.643378,Population_over_65
2,3175.626435,3175.626435,Temperature
3,1419.816152,1419.816152,Fuel_Price


## Try Ridge

In [107]:
ridge_alphas = np.logspace(0, 5, 200)

In [118]:
optimal_ridge = sklearn.linear_model.RidgeCV(alphas=ridge_alphas, cv=10)
optimal_ridge.fit(X_std, y)

RidgeCV(alphas=array([  1.00000e+00,   1.05956e+00, ...,   9.43788e+04,   1.00000e+05]),
    cv=10, fit_intercept=True, gcv_mode=None, normalize=False,
    scoring=None, store_cv_values=False)

In [119]:
print optimal_ridge.alpha_
print optimal_ridge.intercept_

204.907468982
1041361.12476


In [120]:
ridge = sklearn.linear_model.Ridge(alpha=optimal_ridge.alpha_)
ridge_scores = sklearn.model_selection.cross_val_score(ridge, X_std, y,
                                                      cv = 10)
print ridge_scores
print np.mean(ridge_scores)

[ 0.83554436  0.87105612  0.57649641  0.80596189  0.80215809  0.79236993
  0.89356561  0.81511962  0.84713349  0.8342511 ]
0.807365662169


In [121]:
ridge.fit(X_std, y)
print enet.intercept_
ridge_coefs = pd.DataFrame({'variable':X.columns, 'coef':ridge.coef_, 'abs_coef':np.abs(ridge.coef_)})
ridge_coefs.sort_values('abs_coef', inplace=True, ascending=False)
ridge_coefs

1041361.12476


,abs_coef,coef,variable
7,213721.059034,213721.059034,Population
8,130598.845260,130598.845260,Population_over_65
9,100679.833840,100679.833840,Population_under_18
1,97546.437088,97546.437088,Size
0,22624.350020,22624.350020,IsHoliday
5,19636.487642,-19636.487642,CPI
2,17145.320030,-17145.320030,Temperature
6,10067.380092,10067.380092,Average_earning
4,9627.343866,-9627.343866,Unemployment
3,4023.916577,-4023.916577,Fuel_Price


## Try Elastic

In [100]:
l1_ratios = np.linspace(0.01, 1.0, 50)

In [101]:
optimal_enet = sklearn.linear_model.ElasticNetCV(l1_ratio=l1_ratios, 
                                                 n_alphas=100, cv=10)

In [102]:
optimal_enet.fit(X_std,y)

ElasticNetCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
       l1_ratio=array([ 0.01   ,  0.0302 ,  0.05041,  0.07061,  0.09082,  0.11102,
        0.13122,  0.15143,  0.17163,  0.19184,  0.21204,  0.23224,
        0.25245,  0.27265,  0.29286,  0.31306,  0.33327,  0.35347,
        0.37367,  0.39388,  0.41408,  0.43429,  0.45449,  0.47469,
        0.4949 ,  0.515...3837,
        0.85857,  0.87878,  0.89898,  0.91918,  0.93939,  0.95959,
        0.9798 ,  1.     ]),
       max_iter=1000, n_alphas=100, n_jobs=1, normalize=False,
       positive=False, precompute='auto', random_state=None,
       selection='cyclic', tol=0.0001, verbose=0)

In [103]:
print optimal_enet.alpha_
print optimal_enet.l1_ratio_

10599.6837586
1.0


In [104]:
enet = sklearn.linear_model.ElasticNet(alpha=optimal_enet.alpha_, l1_ratio=optimal_enet.l1_ratio_)
enet_scores = sklearn.model_selection.cross_val_score(enet, X_std, y, cv=10)
print enet_scores
print np.mean(enet_scores)

[ 0.86199891  0.88493509  0.5769535   0.76786212  0.80147866  0.79353209
  0.88762845  0.82531839  0.86955148  0.78880846]
0.805806716334


In [105]:
enet.fit(X_std, y)
print enet.intercept_
enet_coefs = pd.DataFrame({'variable':X.columns, 'coef':enet.coef_, 'abs_coef':np.abs(enet.coef_)})
enet_coefs.sort_values('abs_coef', inplace=True, ascending=False)
enet_coefs

1041361.12476


,abs_coef,coef,variable
7,407161.568774,407161.568774,Population
1,81586.027552,81586.027552,Size
9,46186.519484,46186.519484,Population_under_18
2,14977.832676,-14977.832676,Temperature
0,14138.808741,14138.808741,IsHoliday
5,9252.911388,-9252.911388,CPI
3,0.000000,-0.000000,Fuel_Price
4,0.000000,-0.000000,Unemployment
6,0.000000,0.000000,Average_earning
8,0.000000,0.000000,Population_over_65
